# Accredited bodyshops table

In this notebook:
1. We detail how to configure a project, service account and credentials for using Google API to access Drive and Sheets files
2. We download bodyshops data from a spreadsheet maintained by business teams using Google API
3. We curate the table and prepare it for upload
4. We create a table in Postgres to host this data and copy the data into it

---
## 1. Configuring access to Google API

### 1.1. Creating a new service and API configuration

The following steps are mere documentation, skip to `1.2.` to create your Desktop app access:

1. **Create a project in which we enable Google API**: In this process we have create the project `ds-data`. For this project, we have enabled access to `Google Sheets` and `Google Drive` API's. For detail refer to the [official documentation](https://developers.google.com/workspace/guides/create-project)
2. **Create a service account associated with the project**: In this process we hace created the `ds-data-service-account` service account. Data Science team members have been granted access to [this service account](https://console.cloud.google.com/iam-admin/serviceaccounts/details/104289446441655065373;edit=true?project=ds-data-334412)
3. **Create credentials**: Access the [credentials menu for the service account](https://console.cloud.google.com/apis/credentials?project=ds-data-334412). Click on `CREATE CREDENTIALS --> OAuth Client ID`, and select `Desktop app` as application type. By the end of this process we have created the `credential_set1` credentials. Link to [official documentation](https://developers.google.com/workspace/guides/create-credentials).

### 1.2. Enabling access using existing credentials

To enable access spreadsheet data, you need to obtain the `.json` file associated with existing access credentials. To do so:
- Access [the credentials menu](https://console.cloud.google.com/apis/credentials?project=ds-data-334412)
- Cilck on any existing `OAuth 2.0 Client ID` credential
- Click `Download JSON`
- Rename the downloaded file to `credentials.json` and locate it into your root folder

---

## 2. Accessing spreadsheet data

In this section we show how to access existing data.


In [1]:
# Imports
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials


### 2.1. Flow of authorization
Specify scopes of the application we are about to log in as. OAuth authorization for `ds-data-service-account` is capped to read only for both sheets and Drive, but it's good practice to have the scopes limited for your access:


In [2]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

Create credentials object, validating the access of our application:

In [3]:
def obtain_credentials(SCOPES):
    
    ''' Function to collect and return the user credentials object'''

    creds = None
    
    # A token.json file stores user's access and refresh tokens, and is created automatically 
    # when the authorization flow completes for the first time.
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)

    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
            
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(creds.to_json())

    # Return the credentials
    return creds

# Create credentials object
creds = obtain_credentials(SCOPES)


### 2.2. Data download

We first configure the access parameters, which include the following

In [4]:
# Spreadsheet id can be found in the URL to the online spreadsheet
SPREADSHEET_ID = '1A285yoFdMmqsdepQ-rcYC-OJp4vuxqC2yb826qqjmJU'

# Selelct the range of access
RANGE_NAME = 'Oficinas!A1:Q'


Now we can download the spreadsheet data

In [5]:
# Build the service
service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API and collect the results
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,
                            range=RANGE_NAME).execute()
values = result.get('values', [])

# Some diagnostics
print('Total rows collected:', len(values)-1)
print('Columns collected:')
display(values[0])

Total rows collected: 49
Columns collected:


['Nome da Oficina',
 'Razão Social',
 'CNPJ',
 'Telefone',
 'Responsável',
 'Logradouro',
 'Número',
 'Complemento',
 'Bairro',
 'Cidade',
 'Estado',
 'CEP',
 'Full Address',
 'Latitude',
 'Longitude',
 'Horário de recebimento (início)',
 'Horário de recebimento (término)']

---

## 3. Curating the data

Transform to DataFrame

In [6]:
import pandas as pd
data = pd.DataFrame(values[1:], columns=values[0])
data.head()

,Nome da Oficina,Razão Social,CNPJ,Telefone,Responsável,Logradouro,Número,Complemento,Bairro,Cidade,Estado,CEP,Full Address,Latitude,Longitude,Horário de recebimento (início),Horário de recebimento (término)
0,Scattini Funilaria,Scattini Funilaria e Pintura de Autos Ltda,52.644.093/0001-90,(11) 5543-9622,Junior,"Al,. dos Nhambiquaras",1542,,Moema,São Paulo,SP,04090-003,"Al,. dos Nhambiquaras 1542 Moema São Paulo SP ...",-23.6112234,-46.6636607,8:00:00 AM,6:00:00 PM
1,Oficina Marques (Vila Mascote),Oficina Marques Funilaria e Pintura Ltda,07.427.476/0001-52,(11) 5535-1903,Cae,Av. Santa Catarina,2580,,Vila Mascote,São Paulo,SP,04378-200,Av. Santa Catarina 2580 Vila Mascote São Paulo...,-23.6562648,-46.655485,8:00:00 AM,5:00:00 PM
2,Oficina Marques (Santo Amaro),Marques e Marques Funilaria e Pintura Ltda,07.427.481/001-65,(11) 5535-1903,Cae,Av. Santo Amaro,4408,,Brooklin,São Paulo,SP,04556-500,Av. Santo Amaro 4408 Brooklin São Paulo SP 045...,-23.6216384,-46.6845705,8:00:00 AM,5:45:00 PM
3,Ponce e Filho,Ponce & Filho Ltda,54.824.826/0001-02,(11) 2965-7383,Emilio,R. Butiá,42,,Vila Reg. Feijó,São Paulo,SP,03346-010,R. Butiá 42 Vila Reg. Feijó São Paulo SP 03346...,-23.5588674,-46.5703805,8:00:00 AM,5:30:00 PM
4,Dimas Norte,Fiola Reparadora de Veiciulos Eireli,17.237.668/0001-50,(11) 2904-8400,Fiola,Av. Gen. Ataliba Leonel,1727,,Carandiru,São Paulo,SP,02033-010,Av. Gen. Ataliba Leonel 1727 Carandiru São Pau...,-23.4999513,-46.6126825,8:00:00 AM,5:00:00 PM


Clean the data and export

In [7]:
columns = {
    'Nome da Oficina': 'nome_da_oficina',
    'Full Address': 'full_address',
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'CEP': 'cep',
    'Razão Social': 'razao_social',
    'CNPJ': 'cnpj',
    'Telefone': 'telefone',
    'Responsável': 'responsavel',
    'Logradouro': 'logradouro',
    'Número': 'numero',
    'Complemento': 'complemento',
    'Bairro': 'bairro',
    'Cidade': 'cidade',
    'Estado': 'estado',
    'Horário de recebimento (início)': 'inicio_recebimento',
    'Horário de recebimento (término)': 'termino_recebimento'}

# Cleanup
data = data.rename(columns=columns)[columns.values()]
display(data.head())

# Export
data.to_csv('data/bodyshops.csv', index=False)


,nome_da_oficina,full_address,latitude,longitude,cep,razao_social,cnpj,telefone,responsavel,logradouro,numero,complemento,bairro,cidade,estado,inicio_recebimento,termino_recebimento
0,Scattini Funilaria,"Al,. dos Nhambiquaras 1542 Moema São Paulo SP ...",-23.6112234,-46.6636607,04090-003,Scattini Funilaria e Pintura de Autos Ltda,52.644.093/0001-90,(11) 5543-9622,Junior,"Al,. dos Nhambiquaras",1542,,Moema,São Paulo,SP,8:00:00 AM,6:00:00 PM
1,Oficina Marques (Vila Mascote),Av. Santa Catarina 2580 Vila Mascote São Paulo...,-23.6562648,-46.655485,04378-200,Oficina Marques Funilaria e Pintura Ltda,07.427.476/0001-52,(11) 5535-1903,Cae,Av. Santa Catarina,2580,,Vila Mascote,São Paulo,SP,8:00:00 AM,5:00:00 PM
2,Oficina Marques (Santo Amaro),Av. Santo Amaro 4408 Brooklin São Paulo SP 045...,-23.6216384,-46.6845705,04556-500,Marques e Marques Funilaria e Pintura Ltda,07.427.481/001-65,(11) 5535-1903,Cae,Av. Santo Amaro,4408,,Brooklin,São Paulo,SP,8:00:00 AM,5:45:00 PM
3,Ponce e Filho,R. Butiá 42 Vila Reg. Feijó São Paulo SP 03346...,-23.5588674,-46.5703805,03346-010,Ponce & Filho Ltda,54.824.826/0001-02,(11) 2965-7383,Emilio,R. Butiá,42,,Vila Reg. Feijó,São Paulo,SP,8:00:00 AM,5:30:00 PM
4,Dimas Norte,Av. Gen. Ataliba Leonel 1727 Carandiru São Pau...,-23.4999513,-46.6126825,02033-010,Fiola Reparadora de Veiciulos Eireli,17.237.668/0001-50,(11) 2904-8400,Fiola,Av. Gen. Ataliba Leonel,1727,,Carandiru,São Paulo,SP,8:00:00 AM,5:00:00 PM


Check data integrity

In [8]:
data.isna().sum()

nome_da_oficina        0
full_address           0
latitude               0
longitude              0
cep                    0
razao_social           0
cnpj                   0
telefone               0
responsavel            0
logradouro             0
numero                 0
complemento            0
bairro                 0
cidade                 0
estado                 0
inicio_recebimento     0
termino_recebimento    0
dtype: int64

---
## 4. Copy to Postgres

We create a table to hold this data

In [1]:
# Imports
import pandas as pd
import numpy as np
import psycopg2, os

# Read the table and fix the complemento column
data = pd.read_csv('data/bodyshops.csv')
data['complemento'] = data.complemento.fillna('').astype(str)

# Establish connection and create its cursor
try: 
    conn = psycopg2.connect(f"host={os.environ['AURORA_POSTGRES_HOST']} dbname={os.environ['AURORA_POSTGRES_DATABASE']} user={os.environ['AURORA_POSTGRES_USERNAME']} password={os.environ['AURORA_POSTGRES_PWD']}")
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
    

We define data types and create the table if it didn't exist

In [10]:
# Define columns and data types
columns = {
    'nome_da_oficina': 'varchar',
    'full_address': 'varchar',
    'latitude': 'numeric',
    'longitude': 'numeric',
    'cep': 'varchar',
    'razao_social': 'varchar',
    'cnpj': 'varchar',
    'telefone': 'varchar',
    'responsavel': 'varchar',
    'logradouro': 'varchar',
    'numero': 'integer',
    'complemento': 'varchar',
    'bairro': 'varchar',
    'cidade': 'varchar',
    'estado': 'varchar',
    'inicio_recebimento': 'varchar',
    'termino_recebimento': 'varchar'}

# Create the statement to be executed for table creation
create_statement = f"CREATE TABLE IF NOT EXISTS staging_tables.stg_accredited_bodyshops ({', '.join([k+' '+v for k,v in columns.items()])})"

# Now create the table if it doesn't exist
cur.execute(create_statement)
conn.commit()


Truncate the table to remove all previous records - we are updating with the whole spreadsheet

In [11]:
# Now truncate the table, removing previous existing records
cur.execute("TRUNCATE TABLE staging_tables.stg_accredited_bodyshops")
conn.commit()


And finally insert the records in the new table

In [12]:
# Finally copy the local data into the table, which we can do for each row (very few rows for now)
for i in range(data.shape[0]):
    row = data.iloc[i]
    cur.execute(f"INSERT INTO staging_tables.stg_accredited_bodyshops ({', '.join(row.index)}) VALUES ({', '.join(['%s']*len(row))})", \
                tuple([v if i!='numero' else v.item() for i,v in row.items()]))
conn.commit()


Et voila! Check DBeaver to see the created table :)

--- 

#### ANNEX

Checking the snapshot after the the dbt process is finished

In [1]:
# Imports
import pandas as pd
import numpy as np
import psycopg2, os

# Establish connection and create its cursor
try: 
    conn = psycopg2.connect(f"host={os.environ['AURORA_POSTGRES_HOST']} dbname={os.environ['AURORA_POSTGRES_DATABASE']} user={os.environ['AURORA_POSTGRES_USERNAME']} password={os.environ['AURORA_POSTGRES_PWD']}")
    cur = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Postgres database")
    print(e)
    
cur.execute("SELECT * FROM dbt_albertoscf.covered_ceps")
dt = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
dt = dt[['cep', 'reg_susep', 'is_open', 'timestamp_changed']].copy().fillna(0)
dt['is_open'] = dt['is_open'].astype(bool)
dt.head()


,cep,reg_susep,is_open,timestamp_changed
0,00000001,0.0,False,2022-01-13 16:52:17+00:00
1,00000007,0.0,False,2022-01-13 16:52:17+00:00
2,00000011,0.0,False,2022-01-13 16:52:17+00:00
3,00000012,0.0,False,2022-01-13 16:52:17+00:00
4,00000036,0.0,False,2022-01-13 16:52:17+00:00


Get record with maximum timestamp per cep

In [2]:
dt = dt.sort_values('timestamp_changed').groupby('cep').last().reset_index()
dt.head()

,cep,reg_susep,is_open,timestamp_changed
0,0,0.0,True,2022-01-13 16:52:17+00:00
1,00000001,0.0,False,2022-01-13 16:52:17+00:00
2,00000007,0.0,False,2022-01-13 16:52:17+00:00
3,00000011,0.0,False,2022-01-13 16:52:17+00:00
4,00000012,0.0,False,2022-01-13 16:52:17+00:00


In [3]:
pd.DataFrame(dt.groupby(['timestamp_changed', 'is_open']).cep.apply(lambda x: len(x)))


cep
timestamp_changed         is_open        
2022-01-13 16:52:17+00:00 False    107998
                          True      73560

Remove CEP records with SUSEP region = 0 (it means missing is we did `fillna()` up there)

In [4]:
pd.DataFrame(dt.groupby('is_open').reg_susep.apply(lambda x: (x==0).value_counts()))

reg_susep
is_open                 
False   False     106247
        True        1751
True    False      73027
        True         533

In [5]:
dt = dt[dt['reg_susep']!=0].copy().drop(columns=['reg_susep'])

In [6]:
dt.isna().sum()

cep                  0
is_open              0
timestamp_changed    0
dtype: int64

Export

In [7]:
dt.to_csv('2022_01_13_open_ceps.csv', index=False)